<a href="https://colab.research.google.com/github/ravisarojjnu/natural-language-processing/blob/master/Tranlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from more_itertools import flatten
import numpy as np
from polyglot.text import Text

In [0]:
def serializeSentences(sentenceList,reverse=False):
    serializedsentenceList=[]
    for sentence in sentenceList:
        if reverse:
            serialized=Text(reversed(str(sentence).lower())).words
        else:
            serialized=Text(str(sentence).lower()).words
        serializedsentenceList.append(serialized)
    return serializedsentenceList

In [0]:
def createWordIndex(sentenceList):
    wordList = list(set(flatten(sentenceList)))
    try:
        wordList.index(" ")
    except:
        wordList.append(" ")
    wordToIndex = {p: i for i, p in enumerate(wordList)}  #creating an dict that gives each char a number assignment for feeding the char into the model
    return wordToIndex

In [0]:
def createPaddedSequence(serializedSentenceList,wordrToIndex,maxLen=25,value=41):
    sequences = [[wordrToIndex[w] for w in s]for s in serializedSentenceList]
    sequences = pad_sequences(maxlen=maxLen, sequences=sequences, padding="post", value=value)  #changing each letter to its correspondong number
    return sequences  

In [0]:
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from more_itertools import flatten
from sklearn.model_selection import train_test_split
from keras.models import Model, Input,Sequential
from keras.layers import LSTM, GRU,Embedding, Dense, TimeDistributed, Dropout, Bidirectional,concatenate,SpatialDropout1D,Conv1D,Flatten,MaxPooling1D
import pandas as pd
import numpy as np
import nltk
import pickle
import keras

In [22]:
from google.colab import files
uploaded = files.upload()



Saving translatefirstcorp.xlsx to translatefirstcorp.xlsx


In [0]:
import io
df2 = pd.read_excel(io.BytesIO(uploaded['translatefirstcorp.xlsx']),encoding = "utf-8")

In [0]:
df = df2
#print(df)
df["Arabic"]=df["Arabic"].apply(lambda x: x.strip())
df["English"]=df["English"].apply(lambda x: x.strip())

df["Arabic"]=df["Arabic"].apply(lambda x : 'START_ '+ x + ' _END')
df["English"]=df["English"].apply(lambda x : 'START_ '+ x + ' _END')


In [0]:
arabicList=df["Arabic"].tolist()
englishList=df["English"].tolist()

#serializing sentencelist
arabicSeqList=serializeSentences(arabicList)
englishSeqList=serializeSentences(englishList)

arbToIndex=createWordIndex(arabicSeqList)
enToIndex=createWordIndex(englishSeqList)

arbFeed=createPaddedSequence(arabicSeqList,arbToIndex,value=arbToIndex[" "])
enFeed=createPaddedSequence(englishSeqList,enToIndex,value=enToIndex[" "])
enFeed = [to_categorical(i, num_classes=len(enToIndex),dtype="int16") for i in enFeed]
print((enFeed))

In [31]:
arFeedtrain, arFeedtest, enFeedtrain, enFeedtest= train_test_split(arbFeed, enFeed, test_size=0.02) 

enFeedtrain=np.array(enFeedtrain,dtype="int16")

print(enFeedtrain.shape)
model = Sequential()
#model.add(Input(shape=(25,)))
model.add(Embedding(input_dim=len(arbToIndex), output_dim=50, input_length= 25))
model.add(SpatialDropout1D(0.1))
# model.add(LocallyConnected1D(64,1,activation='relu'))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1)))

model.add(Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(len(enToIndex), activation="softmax")))
model.summary()


W0826 13:42:13.037389 140008863479680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 13:42:13.083254 140008863479680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 13:42:13.094901 140008863479680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 13:42:13.124511 140008863479680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0826 13:42:13.135310 

(7480, 25, 10701)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 50)            1247300   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 25, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 256)           183296    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 25, 256)           394240    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 25, 10701)         2750157   
Total params: 4,574,993
Trainable params: 4,574,993
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(arFeedtrain,enFeedtrain,batch_size=4,epochs = 10, validation_split = 0.1, verbose=1) #training



W0826 13:42:20.411916 140008863479680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 13:42:20.444971 140008863479680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0826 13:42:20.668102 140008863479680 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 6732 samples, validate on 748 samples
Epoch 1/10
6732/6732 [==============================] - 391s 58ms/step - loss: 4.3316 - acc: 0.3643 - val_loss: 4.2148 - val_acc: 0.3746
Epoch 2/10
6732/6732 [==============================] - 385s 57ms/step - loss: 4.1526 - acc: 0.3764 - val_loss: 4.1691 - val_acc: 0.3805
Epoch 3/10
6732/6732 [==============================] - 385s 57ms/step - loss: 4.0792 - acc: 0.3863 - val_loss: 4.1432 - val_acc: 0.3855
Epoch 4/10
6732/6732 [==============================] - 384s 57ms/step - loss: 4.0100 - acc: 0.3946 - val_loss: 4.1250 - val_acc: 0.3864
Epoch 5/10
6732/6732 [==============================] - 383s 57ms/step - loss: 3.9491 - acc: 0.4035 - val_loss: 4.1474 - val_acc: 0.3881
Epoch 6/10
 196/6732 [..............................] - ETA: 6:11 - loss: 3.8393 - acc: 0.4171